In [4]:
import json
import pandas as pd
import numpy as np
import os
import sys


import psycopg2 as psy
from psycopg2 import sql
from io import StringIO
import sqlalchemy
from sqlalchemy import create_engine

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/src")
    
from mtg_helpers import *

con_details = {"dbname" : 'pay_to_play', 
               "user" : os.environ['PGSQL_P_USER'], 
               "password" : os.environ['PGSQL_P_PWD'], 
               "host" : 'localhost'} 

In [204]:
data_path = '../data/MTGJSONcards/'
file_name = 'WAR.json'
file = open(data_path+file_name, 'r')
card_data = json.load(file)['data']


In [205]:
print(card_data.keys())

dict_keys(['baseSetSize', 'block', 'booster', 'cards', 'code', 'isFoilOnly', 'isOnlineOnly', 'keyruneCode', 'mcmId', 'mcmIdExtras', 'mcmName', 'mtgoCode', 'name', 'releaseDate', 'tcgplayerGroupId', 'tokens', 'totalSetSize', 'translations', 'type'])


In [206]:
set_details = pd.DataFrame({'setcode':card_data['code'], 'setname':card_data['name'], 'setreleasedate': card_data['releaseDate']}, index=[0])
#set_details = pd.DataFrame(data = set_data.values(), columns = set_data.keys())
add_new_card_data(con_details, set_details, "setdetails")
#set_details

Successful updating of setdetails


In [208]:
types = [int, float, bool, str]
dict_peek(card_data, types)

Key baseSetSize has values of <class 'int'> type with a value of 264
Key block has values of <class 'str'> type with a value of Guilds of Ravnica
Key booster has values of <class 'dict'> type with a length of 2
Key cards has values of <class 'list'> type with a length of 311
Key code has values of <class 'str'> type with a value of WAR
Key isFoilOnly has values of <class 'bool'> type with a value of False
Key isOnlineOnly has values of <class 'bool'> type with a value of False
Key keyruneCode has values of <class 'str'> type with a value of WAR
Key mcmId has values of <class 'int'> type with a value of 2439
Key mcmIdExtras has values of <class 'int'> type with a value of 2451
Key mcmName has values of <class 'str'> type with a value of War of the Spark
Key mtgoCode has values of <class 'str'> type with a value of WAR
Key name has values of <class 'str'> type with a value of War of the Spark
Key releaseDate has values of <class 'str'> type with a value of 2019-05-03
Key tcgplayerGroupId

In [209]:
for k, v in card_data.items():
    if type(v) == dict:
        print("Let's explore " + k)
        dict_peek(v, types)
        

Let's explore booster
Key arena has values of <class 'dict'> type with a length of 4
Key default has values of <class 'dict'> type with a length of 3
Let's explore translations
Key Chinese Simplified has values of <class 'NoneType'> type with a value of None
Key Chinese Traditional has values of <class 'NoneType'> type with a value of None
Key French has values of <class 'str'> type with a value of La Guerre des Planeswlakers
Key German has values of <class 'NoneType'> type with a value of None
Key Italian has values of <class 'NoneType'> type with a value of None
Key Japanese has values of <class 'NoneType'> type with a value of None
Key Korean has values of <class 'NoneType'> type with a value of None
Key Portuguese (Brazil) has values of <class 'NoneType'> type with a value of None
Key Russian has values of <class 'NoneType'> type with a value of None
Key Spanish has values of <class 'NoneType'> type with a value of None


In [210]:
dict_peek(card_data['booster']['arena'], types)

Key boosters has values of <class 'list'> type with a length of 2
Key boostersTotalWeight has values of <class 'int'> type with a value of 121
Key name has values of <class 'str'> type with a value of War of the Spark Arena Booster
Key sheets has values of <class 'dict'> type with a length of 5


In [212]:
dict_peek(card_data['booster']['default'], types)

Key boosters has values of <class 'list'> type with a length of 4
Key boostersTotalWeight has values of <class 'int'> type with a value of 4840
Key sheets has values of <class 'dict'> type with a length of 7


In [214]:
#explore cards and tokens
print(card_data['cards'][:3])

[{'artist': 'Wisnu Tan', 'availability': ['arena', 'mtgo', 'paper'], 'borderColor': 'black', 'colorIdentity': [], 'colors': [], 'convertedManaCost': 4.0, 'edhrecRank': 1100, 'foreignData': [{'language': 'German', 'multiverseId': 461192, 'name': 'Karn, der Große Schöpfer', 'text': 'Aktivierte Fähigkeiten von Artefakten, die deine Gegner kontrollieren, können nicht aktiviert werden.\n+1: Bis zu deinem nächsten Zug wird bis zu ein Nichtkreatur-Artefakt deiner Wahl zu einer Artefaktkreatur mit Stärke und Widerstandskraft jeweils gleich ihrer umgewandelten Manakosten.\n−2: Du kannst eine Artefaktkarte bestimmen, die du besitzt und die sich außerhalb der Partie oder im Exil befindet, sie offen vorzeigen und auf deine Hand nehmen.', 'type': 'Legendärer Planeswalker — Karn'}, {'language': 'Spanish', 'multiverseId': 461456, 'name': 'Karn, el Gran Creador', 'text': 'Las habilidades activadas de los artefactos que controlan tus oponentes no pueden activarse.\n+1: Hasta tu próximo turno, hasta un 

In [215]:
cards = pd.DataFrame(card_data['cards'])

In [216]:
cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 44 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   artist             311 non-null    object 
 1   availability       311 non-null    object 
 2   borderColor        311 non-null    object 
 3   colorIdentity      311 non-null    object 
 4   colors             311 non-null    object 
 5   convertedManaCost  311 non-null    float64
 6   edhrecRank         296 non-null    float64
 7   foreignData        311 non-null    object 
 8   frameVersion       311 non-null    object 
 9   hasFoil            311 non-null    bool   
 10  hasNonFoil         311 non-null    bool   
 11  identifiers        311 non-null    object 
 12  layout             311 non-null    object 
 13  leadershipSkills   91 non-null     object 
 14  legalities         311 non-null    object 
 15  loyalty            75 non-null     object 
 16  manaCost           288 non

In [217]:
cards.shape

(311, 44)

In [219]:

#define standard schema for postgreSQL database
prim_key = 'uuid'
core_fields = ['uuid','name', 'rarity', 'setCode', 'types', 'keywords', 'colorIdentity', 'colors', 'convertedManaCost', 'loyalty', 'manaCost', 'subtypes', 'supertypes', 'text', 'type', 'power', 'toughness']
demand_fields = ['uuid','edhrecRank']
identifiers_fields = ['uuid','identifiers']
detritus_fields = ['uuid','number', 'frameEffects', 'artist', 'availability', 'borderColor', 'foreignData', 'frameVersion', 'hasFoil', 'hasNonFoil', 'isReprint', 'layout', 'leadershipSkills', 'legalities', 'printings', 'purchaseUrls', 'rulings', 'variations', 'flavorText', 'isPromo', 'isStarter', 'promoTypes', 'watermark', 'originalText', 'originalType']

pre_defined_fields = set(core_fields + demand_fields + identifiers_fields + detritus_fields)
data_set_fields = set(cards.columns)

gap_type, diff_fields = check_column_completeness(pre_defined_fields, data_set_fields) 
print(gap_type)
print(diff_fields)


if gap_type == 'missing':
    cards[list(diff_fields)] = np.nan
elif gap_type == 'novel':
    print(cards[list(diff_fields)].head())



WARNING - THERE ARE NOVEL FIELDS IN THE CURRENT CARD DATA SET.
novel
{'isStorySpotlight', 'isAlternative'}
  isStorySpotlight isAlternative
0              NaN           NaN
1              NaN           NaN
2              NaN           NaN
3              NaN           NaN
4              NaN           NaN


In [222]:
#
#workaround until better trapping for both missing and novel fields.
#cards['watermark'] = np.nan
#


id_fields_dicts = ['identifiers']
detritus_fields_dicts = ['foreignData', 'rulings', 'leadershipSkills', 'legalities', 'purchaseUrls']
add_detritus = input('Ok to add novel fields to detritus? y or n').lower()
if add_detritus == 'y':
    detritus_fields_dicts.append(list(diff_fields))

core = cards[core_fields]
demand = cards[demand_fields]

detritus = build_combo_data(detritus_fields, detritus_fields_dicts, cards)
identifiers = build_combo_data(identifiers_fields, id_fields_dicts, cards)

tables = {"core":core, "demand":demand, "identifiers":identifiers, "detritus":detritus}

Ok to add novel fields to detritus? y or nn


In [223]:
detritus.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   uuid              311 non-null    object 
 1   number            311 non-null    object 
 2   frameEffects      21 non-null     object 
 3   artist            311 non-null    object 
 4   availability      311 non-null    object 
 5   borderColor       311 non-null    object 
 6   foreignData       311 non-null    object 
 7   frameVersion      311 non-null    object 
 8   hasFoil           311 non-null    bool   
 9   hasNonFoil        311 non-null    bool   
 10  isReprint         28 non-null     object 
 11  layout            311 non-null    object 
 12  leadershipSkills  311 non-null    object 
 13  legalities        311 non-null    object 
 14  printings         311 non-null    object 
 15  purchaseUrls      311 non-null    object 
 16  rulings           311 non-null    object 
 1

In [224]:
detritus[['isReprint', 'layout', 'leadershipSkills', 'legalities', 'printings']]

,isReprint,layout,leadershipSkills,legalities,printings
0,NaN,normal,"{'brawl': False, 'commander': False, 'oathbrea...","{'commander': 'Legal', 'duel': 'Legal', 'gladi...","[PRM, PWAR, SLD, WAR]"
1,True,normal,nan,"{'commander': 'Legal', 'duel': 'Legal', 'gladi...","[C13, DDH, IMA, M11, M15, M19, MB1, WAR]"
2,NaN,normal,nan,"{'commander': 'Legal', 'duel': 'Legal', 'gladi...","[JMP, WAR]"
3,NaN,normal,nan,"{'commander': 'Legal', 'duel': 'Legal', 'gladi...","[JMP, MB1, WAR]"
4,NaN,normal,nan,"{'commander': 'Legal', 'duel': 'Legal', 'gladi...",[WAR]
...,...,...,...,...,...
306,NaN,normal,"{'brawl': False, 'commander': False, 'oathbrea...","{'commander': 'Legal', 'duel': 'Legal', 'gladi...","[PRM, PWAR, SLD, WAR]"
307,NaN,normal,"{'brawl': False, 'commander': False, 'oathbrea...","{'commander': 'Legal', 'duel': 'Legal', 'gladi...","[PRM, PWAR, SLD, WAR]"
308,NaN,normal,"{'brawl': False, 'commander': False, 'oathbrea...","{'commander': 'Legal', 'duel': 'Legal', 'gladi...","[PPRO, PRM, PWAR, SLD, WAR]"
309,NaN,normal,"{'brawl': False, 'commander': False, 'oathbrea...","{'commander': 'Legal', 'duel': 'Legal', 'gladi...","[PRM, PWAR, SLD, WAR]"


In [225]:
conn = connect(con_details)

Connecting to PostgreSQL database...
Connection successful


In [226]:
for table, data in tables.items():
    add_new_card_data(con_details, data, table)

Successful updating of core
Successful updating of demand
Successful updating of identifiers
Successful updating of detritus


In [227]:
core.head()

,uuid,name,rarity,setCode,types,keywords,colorIdentity,colors,convertedManaCost,loyalty,manaCost,subtypes,supertypes,text,type,power,toughness
0,0a679f46-9f50-5ac1-a6bf-c802c67fc0a0,"Karn, the Great Creator",rare,WAR,[Planeswalker],NaN,[],[],4.0,5,{4},[Karn],[Legendary],Activated abilities of artifacts your opponent...,Legendary Planeswalker — Karn,NaN,NaN
1,d451691b-6cc7-5ea7-968b-2abae1beb622,Ajani's Pridemate,uncommon,WAR,[Creature],NaN,[W],[W],2.0,NaN,{1}{W},"[Cat, Soldier]",[],"Whenever you gain life, put a +1/+1 counter on...",Creature — Cat Soldier,2,2
2,470332c9-e7d4-5b11-b0ca-4c9f877f8e5c,Battlefield Promotion,common,WAR,[Instant],NaN,[W],[W],2.0,NaN,{1}{W},[],[],Put a +1/+1 counter on target creature. That c...,Instant,NaN,NaN
3,4b60a0b8-ddee-5a88-a578-22fb009c117e,Bulwark Giant,common,WAR,[Creature],NaN,[W],[W],6.0,NaN,{5}{W},"[Giant, Soldier]",[],"When Bulwark Giant enters the battlefield, you...",Creature — Giant Soldier,3,6
4,01013cab-be73-532f-b6bf-0f24d0c3792f,Enforcer Griffin,common,WAR,[Creature],[Flying],[W],[W],5.0,NaN,{4}{W},[Griffin],[],Flying,Creature — Griffin,3,4
